In [0]:
# This notebook shows how to mount your adls account into your databricks file system

In [0]:
storage_account = "aldlsj2dtechstorage"
container_name = "j2dadlscontainer"
mount_path = "/mnt/my_container"

In [0]:
# accessing secrets from scret scope 
client_id = dbutils.secrets.get(scope="j2d-secret-scope-vault", key="adb-05-client-id")
tenant_id = dbutils.secrets.get(scope="j2d-secret-scope-vault", key="adb-05-tenent-id")
secret_id = dbutils.secrets.get(scope="j2d-secret-scope-vault", key="adb-05-secret-id")

In [0]:
configs = {
    "fs.azure.account.auth.type": "OAuth",
    "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
    "fs.azure.account.oauth2.client.id": client_id,
    "fs.azure.account.oauth2.client.secret": secret_id,
    "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"
}

In [0]:
dbutils.fs.mounts()

In [0]:
# dbutils.fs.unmount("/mnt/my_adls_container")

In [0]:
dbutils.fs.mounts()

In [0]:
if [mount_path for mountInfo in dbutils.fs.mounts() if mountInfo.mountPoint == mount_path]:
    print(f"Error mounting {container_name} - it exists already")
else:
    dbutils.fs.mount(
        source = f"abfss://{container_name}@{storage_account}.dfs.core.windows.net/",
        mount_point = mount_path,
        extra_configs = configs
    )

In [0]:
dbutils.fs.mounts()

In [0]:
# read a csv file from adls path

emp_df = (
    spark.read.format("csv")
    .option("header", True)
    .load("/mnt/my_container/raw_datasets/Employee.csv")
)
display(emp_df)

In [0]:
dbutils.fs.ls("/mnt/my_container/")

In [0]:
%fs
ls /mnt/my_container

In [0]:
emp_df.write.format("parquet").mode("overwrite").save("/mnt/my_container/ready_datasets/Employee")